In [1]:
import os
import pandas as pd
from tqdm import tqdm

In [4]:
filename = "YOUR_DIRECTORY/touchevent.csv"

In [33]:
def decoupling_time(r):
    '''
    This will output in this format: 1970-08-22 19:25:00 
    '''
    s = str(r)
    year, month, day, hour, minute, second = s[:4], s[4:6], s[6:8], s[8:10], s[10:12], s[12:14]
    
    return '-'.join([year, month, day]) + ' ' + ':'.join([hour, minute, second])
#decoupling_time(20201117235750093)

'2020-11-17 23:57:50'

In [76]:
final_data = pd.DataFrame()

row_count = 10000
c = 0
for chunk in tqdm(pd.read_csv(filename, chunksize=row_count)): 
    #deep copy 
    temp = pd.DataFrame(columns=['experimentid', 'userid'])
    temp = chunk[['experimentid', 'userid']]
    
    #turning now time from raw to machine readable ; #every minute has a specific count                                         
    shortened_times = pd.to_datetime(chunk['timestamp'].apply(decoupling_time), yearfirst=True).dt.floor('min')
    temp.insert(2, 'timestamp', shortened_times)
    
    #grouping 
    df_grouped = temp.groupby(['userid', 'experimentid', 'timestamp']).size()                                         
    df_grouped = df_grouped.reset_index(name='touches')
    
    #appending to a global df
    final_data = pd.concat([final_data, df_grouped], ignore_index=True) 
    
    if c == 100:
        break
    c += 1

100it [00:03, 26.83it/s]


In [77]:
final_data.head()

,userid,experimentid,timestamp,touches
0,3,wenetItaly,2020-11-27 15:04:00,62
1,3,wenetItaly,2020-11-27 15:05:00,48
2,3,wenetItaly,2020-11-27 15:06:00,97
3,3,wenetItaly,2020-11-27 15:07:00,211
4,3,wenetItaly,2020-11-27 15:08:00,352


In [71]:
# SAVING the file
final_data.to_csv('reduced_file.csv', index=False)